In [39]:
import pandas as pd
from pathlib import Path
import calendar

file_path=Path("Resources/fire_data/mapdataall.csv")
df=pd.read_csv(file_path, encoding="utf-8")

df.head()

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_administrative_unit_url,incident_county,incident_location,incident_acres_burned,incident_containment,...,incident_latitude,incident_type,incident_id,incident_url,incident_date_extinguished,incident_dateonly_extinguished,incident_dateonly_created,is_active,calfire_incident,notification_desired
0,Bridge Fire,Y,2018-01-09T13:46:00Z,2017-10-31T11:22:00Z,Shasta-Trinity National Forest,NaN,Shasta,"I-5 and Turntable Bay, 7 miles NE of Shasta Lake",37.0,100.0,...,40.774000,NaN,2ca11d45-8139-4c16-8af0-880d99b21e82,https://incidentupdate.fire.ca.gov/incidents/2...,2018-01-09T13:46:00Z,2018-01-09,2017-10-31,N,False,False
1,Pala Fire,Y,2020-09-16T14:07:35Z,2009-05-24T14:56:00Z,CAL FIRE San Diego Unit,NaN,San Diego,"Hwy 76 and Pala Temecula, northwest of Pala",122.0,100.0,...,1.000000,Wildfire,8f61f461-552d-4538-b186-35ab030da416,https://incidentupdate.fire.ca.gov/incidents/2...,2009-05-25T00:00:00Z,2009-05-25,2009-05-24,N,True,False
2,River Fire,Y,2022-10-24T11:39:23Z,2013-02-24T08:16:00Z,CAL FIRE San Bernardino Unit,NaN,Inyo,"south of Narrow Gauge Rd & north of Hwy 136, e...",407.0,100.0,...,36.602575,NaN,094719ba-a47b-4abb-9ec5-a506b2b9fd23,https://incidentupdate.fire.ca.gov/incidents/2...,2013-02-28T20:00:00Z,2013-02-28,2013-02-24,N,True,False
3,Fawnskin Fire,Y,2013-04-22T09:00:00Z,2013-04-20T17:30:00Z,San Bernardino National Forest,NaN,San Bernardino,"west of Delamar Mountain, north of the communi...",30.0,100.0,...,34.288877,NaN,58f89ff8-bd3e-4355-b1c0-8fa05c747d3f,https://incidentupdate.fire.ca.gov/incidents/2...,2013-04-22T09:00:00Z,2013-04-22,2013-04-20,N,False,False
4,Gold Fire,Y,2013-05-01T07:00:00Z,2013-04-30T12:59:00Z,CAL FIRE Madera-Mariposa-Merced Unit,NaN,Madera,Between Road 210 and Road 200 near Fine Gold C...,274.0,100.0,...,37.116295,NaN,357ffc13-bef9-48eb-810f-c5de851972eb,https://incidentupdate.fire.ca.gov/incidents/2...,2013-05-01T07:00:00Z,2013-05-01,2013-04-30,N,True,False


In [11]:
#gather list of original columns
#list(df.columns)

In [42]:
#create new data frame with select columns
fire_df = df[['incident_name',
 'incident_administrative_unit',
 'incident_county',
 'incident_acres_burned',
 'incident_longitude',
 'incident_latitude',
 'incident_dateonly_extinguished',
 'incident_dateonly_created'
]]

#rename columns
fire_df = fire_df.rename(columns={
    'incident_name': "Name",
     'incident_administrative_unit': "Admin unit",
     'incident_county': "County",
     'incident_acres_burned': "Acres burned" ,
     'incident_longitude': "Lon",
     'incident_latitude': "Lat",
     'incident_dateonly_extinguished': "Date extinguished",
     'incident_dateonly_created': "Date created"
})

#fire_incidents_df.head()
fire_df.head(50)

,Name,Admin unit,County,Acres burned,Lon,Lat,Date extinguished,Date created
0,Bridge Fire,Shasta-Trinity National Forest,Shasta,37.0,-122.309000,40.774000,2018-01-09,2017-10-31
1,Pala Fire,CAL FIRE San Diego Unit,San Diego,122.0,1.000000,1.000000,2009-05-25,2009-05-24
2,River Fire,CAL FIRE San Bernardino Unit,Inyo,407.0,-118.016510,36.602575,2013-02-28,2013-02-24
3,Fawnskin Fire,San Bernardino National Forest,San Bernardino,30.0,-116.941311,34.288877,2013-04-22,2013-04-20
4,Gold Fire,CAL FIRE Madera-Mariposa-Merced Unit,Madera,274.0,-119.635004,37.116295,2013-05-01,2013-04-30
5,Panther Fire,CAL FIRE Butte Unit,Tehama,6896.0,-121.595555,40.190062,2013-05-09,2013-05-01
6,Silverado Fire,CAL FIRE Sonoma-Lake-Napa Unit,Napa,75.0,-122.350844,38.441792,2013-05-01,2013-04-30
7,Yellow Fire,CAL FIRE Sonoma-Lake-Napa Unit,Sonoma,125.0,-122.655616,38.638828,2013-05-03,2013-05-01
8,Summit Fire,CAL FIRE/Riverside County Fire,Riverside,2956.0,-116.941311,34.288877,2013-05-04,2013-05-01
9,Tres Pinos Fire,CAL FIRE San Benito-Monterey,San Benito,354.0,-120.937494,37.160346,2013-05-03,2013-05-03


In [49]:
#drop rows where county and acres burned columns are null
fire_incidents = fire_df.dropna(subset=['Acres burned','County', 'Date extinguished'])

#convert to datetime
fire_incidents['Date extinguished'] = pd.to_datetime(fire_incidents.loc[:,'Date extinguished'])
fire_incidents['Date created'] = pd.to_datetime(fire_incidents.loc[:,'Date created'])

#calculate duration
fire_incidents['Duration (days)'] = fire_incidents.loc[:,'Date extinguished'] - fire_incidents.loc[:,'Date created']

#add years columns. Note dates are currently in datetime
fire_incidents["Year extinguished"]=fire_incidents.loc[:,"Date extinguished"].dt.year
fire_incidents["Year"]=fire_incidents.loc[:,"Date created"].dt.year

#add months column by name (& number?) do not need extinguished only created
fire_incidents["Month created (num)"]=fire_incidents["Date created"].dt.month
fire_incidents["Month"]=fire_incidents["Month created (num)"].apply(lambda x: calendar.month_name[x])

##***INVESTIGATE HOW TO GET RID OF WARNINGS

fire_incidents

/var/folders/pp/jydch3xn6ys5qzxqnlsfwm180000gn/T/ipykernel_16060/972722752.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fire_incidents['Date extinguished'] = pd.to_datetime(fire_incidents.loc[:,'Date extinguished'])
/var/folders/pp/jydch3xn6ys5qzxqnlsfwm180000gn/T/ipykernel_16060/972722752.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fire_incidents['Date created'] = pd.to_datetime(fire_incidents.loc[:,'Date created'])
/var/folders/pp/jydch3xn6ys5qzxqnlsfwm180000gn/T/ipykernel_16060/972722752.p

,Name,Admin unit,County,Acres burned,Lon,Lat,Date extinguished,Date created,Duration (days),Year extinguished,Year,Month created (num),Month
0,Bridge Fire,Shasta-Trinity National Forest,Shasta,37.0,-122.309000,40.774000,2018-01-09,2017-10-31,70 days,2018,2017,10,October
1,Pala Fire,CAL FIRE San Diego Unit,San Diego,122.0,1.000000,1.000000,2009-05-25,2009-05-24,1 days,2009,2009,5,May
2,River Fire,CAL FIRE San Bernardino Unit,Inyo,407.0,-118.016510,36.602575,2013-02-28,2013-02-24,4 days,2013,2013,2,February
3,Fawnskin Fire,San Bernardino National Forest,San Bernardino,30.0,-116.941311,34.288877,2013-04-22,2013-04-20,2 days,2013,2013,4,April
4,Gold Fire,CAL FIRE Madera-Mariposa-Merced Unit,Madera,274.0,-119.635004,37.116295,2013-05-01,2013-04-30,1 days,2013,2013,4,April
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2192,South Fire,Ventura County Fire Department,Ventura,2715.0,-118.993800,34.327000,2023-12-12,2023-12-09,3 days,2023,2023,12,December
2193,Pine Fire,Kern County Fire Department,Kern,181.0,-118.869600,35.559300,2023-12-15,2023-12-15,0 days,2023,2023,12,December
2194,Border 42 Fire,CAL FIRE San Diego Unit,San Diego,50.0,-116.771800,32.603600,2023-12-22,2023-12-21,1 days,2023,2023,12,December
2195,Palmas Fire,Bureau of Land Management,Riverside,239.0,-115.827840,33.510379,2024-02-26,2024-02-20,6 days,2024,2024,2,February


In [23]:
fire_incidents.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1943 entries, 0 to 2196
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   Name                 1943 non-null   object         
 1   Admin unit           1934 non-null   object         
 2   County               1943 non-null   object         
 3   Acres burned         1943 non-null   float64        
 4   Lon                  1943 non-null   float64        
 5   Lat                  1943 non-null   float64        
 6   Date extinguished    1943 non-null   datetime64[ns] 
 7   Date created         1943 non-null   datetime64[ns] 
 8   Duration (days)      1943 non-null   timedelta64[ns]
 9   Year extinguished    1943 non-null   int32          
 10  Year                 1943 non-null   int32          
 11  Month created (num)  1943 non-null   int32          
 12  Month                1943 non-null   object         
dtypes: datetime64[ns](2), f

In [24]:
fire_incidents.to_csv("Resources/fire_data/fire_all_clean.csv", index_label="City_ID")

In [48]:
#county key metrics

#groupby county
county_group = fire_incidents.groupby(["Month","County"]) #ADD GROUP BY MONTH FIRST

#create summary data frame per county
county_summary = county_group.agg(
    #count incidents
    Incident_count=("County","count"), 
    
    #total acre burned
    Total_acres_burned=("Acres burned", "sum"),
    
    #avg acres burned and round to whole number
    Avg_acres_burned=("Acres burned", "mean"),
    
    #avg duration and only exctract days as an integer  
    Avg_duration=("Duration (days)", lambda x: x.mean().days)
)

# convert 'total_acres_burned' and 'avg_acres_burned' to whole numbers/integer
county_summary['Total_acres_burned'] = county_summary['Total_acres_burned'].astype(int)
county_summary['Avg_acres_burned'] = county_summary['Avg_acres_burned'].round(0).astype(int)

#sort by descending incident count
#county_summary = county_summary.sort_values(by=["incident_count"], ascending=False)

#preview
county_summary.head(25)

#county_summary.info()
county_summary = county_summary.reset_index()
county_summary

,Month,County,Incident_count,Total_acres_burned,Avg_acres_burned,Avg_duration
0,April,Butte,1,10,10,8
1,April,Contra Costa,1,47,47,0
2,April,Fresno,4,5438,1360,260
3,April,Humboldt,1,0,0,254
4,April,Imperial,1,275,275,8
...,...,...,...,...,...,...
405,September,Trinity,5,15727,3145,52
406,September,Tulare,2,19034,9517,60
407,September,Tuolumne,6,1953,326,53
408,September,Ventura,1,279,279,4


In [18]:
# year key metrics

#groupby year created
year_group = fire_incidents.groupby(["Year"])

#create data frame with incident count and acres burned
year_summary = year_group.agg(
    Incident_count=("County","count"), 
    Total_acres_burned=("Acres burned", "sum")
)

#sort by descending incident count
year_created_summary = year_summary.sort_values(by=["Year"], ascending=False)

#preview
year_created_summary.head(25)

,Incident_count,Total_acres_burned
Year,,
2024,2,250.0
2023,93,310026.0
2022,84,140415.0
2021,157,2289096.0
2020,205,2453742.0
2019,205,200634.0
2018,300,1529897.0
2017,427,1258294.0
2016,155,452101.0


In [19]:
#month key metrics

#groupby month created 
month_group = fire_incidents.groupby(["Month"])

#create data frame with incident count and acres burned
month_summary = month_group.agg(
    Incident_count=("County","count"),
    #incident_avg=("county", "mean"),
    Total_acres_burned=("Acres burned", "sum"),
    Avg_acres_burned=("Acres burned", "mean"),
    County_count=("County", "nunique")
)

#sort by descending incident count
month_summary = month_summary.sort_values(by=["Incident_count"], ascending=False)

#preview
month_summary.head(25)

,Incident_count,Total_acres_burned,Avg_acres_burned,County_count
Month,,,,
July,478,3749362.0,7843.853556,66
June,427,412379.0,965.758782,56
August,354,3852007.0,10881.375706,61
September,220,668800.0,3040.000000,54
May,165,125753.0,762.139394,39
October,150,408635.0,2724.233333,46
April,44,14373.0,326.659091,24
November,43,272865.0,6345.697674,24
December,27,321923.0,11923.074074,14
